In [17]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
%autoreload 2

In [75]:
from lib import sim_env as sim
from lib import loads
from lib.utils import DTParser
from lib import agents
import datetime

In [76]:
dt_parser = DTParser()

In [77]:
dt = datetime.datetime(2023, 11, 21)
dt

datetime.datetime(2023, 11, 21, 0, 0)

In [81]:
env = sim.SimulationEnvironment(name='base', time_unit='seconds', starting_datetime=dt)
job_generator = loads.RandomJobGenerator(seed=2)
infstruct_mgr = sim.InfrastructureManager(env=env)
dispatcher = sim.Dispatcher(env=env, priority_rule='FIFO')

INFO:env:Successfully registered Infrastructure Manager in Env >>base<<
INFO:env:Successfully registered Dispatcher in Env >>base<<


In [82]:
# source
area_source = sim.ProductionArea(env=env, custom_identifier=1000)
group_source = sim.StationGroup(env=env, custom_identifier=1000)
area_source.add_subsystem(group_source)
td = dt_parser.timedelta_from_val(val=2., time_unit='hours')
source = sim.Source(env=env, custom_identifier='source', proc_time=td, 
                random_generation=True, job_generator=job_generator, num_gen_jobs=5)
group_source.add_subsystem(source)

# sink
area_sink = sim.ProductionArea(env=env, custom_identifier=2000)
group_sink = sim.StationGroup(env=env, custom_identifier=2000)
area_sink.add_subsystem(group_sink)
sink = sim.Sink(env=env, custom_identifier='sink')
group_sink.add_subsystem(sink)

# processing stations
# prod area 1
area_prod = sim.ProductionArea(env=env, custom_identifier=1)
group_prod = sim.StationGroup(env=env, custom_identifier=1)
area_prod.add_subsystem(group_prod)
group_prod2 = sim.StationGroup(env=env, custom_identifier=2)
area_prod.add_subsystem(group_prod2)
# prod area 2
#area_prod2 = ProductionArea(env=env, custom_identifier=2)
#group_prod3 = StationGroup(env=env, custom_identifier=3)
#area_prod2.add_subsystem(group_prod3)
#area_prod.add_subsystem(group_prod3)
## machines
for machine in range(3):
    buffer = sim.Buffer(capacity=20, env=env, custom_identifier=(10+machine))
    if machine == 5:
        MachInst = sim.Machine(env=env, custom_identifier=machine, buffers=[buffer], setup_time=5.)
    else:
        MachInst = sim.Machine(env=env, custom_identifier=machine, buffers=[buffer])
        
    if machine == 0:
        testMachInst = MachInst
    
    if machine < 2:
        group_prod.add_subsystem(buffer)
        group_prod.add_subsystem(MachInst)
    elif machine >= 2:
        group_prod2.add_subsystem(buffer)
        group_prod2.add_subsystem(MachInst)
    else:
        pass
        #group_prod3.add_subsystem(buffer)
        #group_prod3.add_subsystem(MachInst)
    

add_machine_to_bottleneck: bool = False
if add_machine_to_bottleneck:
    buffer = sim.Buffer(capacity=20, env=env, custom_identifier=(10+machine+1))
    MachInst = sim.Machine(env=env, custom_identifier=machine+1, buffers=[buffer])
    group_prod3.add_subsystem(buffer)
    group_prod3.add_subsystem(MachInst)

INFO:infstrct:Successfully registered object with SystemID 0 and name ProductionArea_env_0
INFO:infstrct:Successfully registered object with SystemID 0 and name StationGroup_env_0
INFO:infstrct:Successfully added System (type: StationGroup, custom_id: 1000, name: StationGroup_env_0) to System (type: ProductionArea, custom_id: 1000, name: ProductionArea_env_0).
INFO:infstrct:Successfully registered object with SystemID 0 and name Source_env_0
INFO:infstrct:Successfully added System (type: Resource, custom_id: source, name: Source_env_0) to System (type: StationGroup, custom_id: 1000, name: StationGroup_env_0).
INFO:infstrct:Successfully registered object with SystemID 1 and name ProductionArea_env_1
INFO:infstrct:Successfully registered object with SystemID 1 and name StationGroup_env_1
INFO:infstrct:Successfully added System (type: StationGroup, custom_id: 2000, name: StationGroup_env_1) to System (type: ProductionArea, custom_id: 2000, name: ProductionArea_env_1).
INFO:infstrct:Succes

In [83]:
alloc_agent = agents.AllocationAgent(assoc_system=area_prod)

INFO:env:Successfully registered Allocation Agent in System (type: ProductionArea, custom_id: 1, name: ProductionArea_env_2)


In [84]:
alloc_agent._assoc_infstrct_objs

(System (type: Resource, custom_id: 0, name: Machine_env_3),
 System (type: Resource, custom_id: 1, name: Machine_env_5),
 System (type: Resource, custom_id: 2, name: Machine_env_7))